### Projekt: Autoencoder + Klasteryzacja + Inpainting (szkielet)

Importy

In [ ]:
from comet_ml import Experiment
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from sklearn.cluster import KMeans
import umap
import matplotlib.pyplot as plt
import numpy as np
import os

Importy z lokalnych plików

In [ ]:
from src.models import ConvAutoencoder
from src.data import MaskedDataset, random_mask, rectangular_mask, noise_mask
from src.utils import train_autoencoder, extract_latent_vectors, cluster_and_visualize, visualize_reconstructions

Konfiguracja Comet ML

In [ ]:
experiment = Experiment(
    api_key="TWOJ_API_KEY_TUTAJ",  # ← wklej swój klucz
    project_name="inpainting-autoencoder",
    workspace="twoj_login_comet"    # np. 'lukasz_piotrek'
)

Przygotowanie danych

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
masked_data = MaskedDataset(train_data)
train_loader = DataLoader(masked_data, batch_size=32, shuffle=True)

Konfiguracja modelu i trenowanie

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvAutoencoder(latent_dim=128).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print(f"Używane urządzenie: {device}")
print(f"Liczba parametrów modelu: {sum(p.numel() for p in model.parameters()):,}")

Logowanie parametrów do Comet ML

In [ ]:
experiment.log_parameters({
    "epochs": 5,
    "batch_size": 32,
    "learning_rate": 1e-3,
    "latent_dim": 128,
    "dataset": "CIFAR10",
    "device": str(device)
})

Trenowanie modelu

In [ ]:
losses = train_autoencoder(
    model=model,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    epochs=5,
    experiment=experiment
)

Wizualizacja rekonstrukcji (przykładowych)

In [ ]:
visualize_reconstructions(model, train_loader, device, n_samples=6)

Klasteryzacja przestrzeni latentnej

- Ekstraktowanie wektorów latentnych
- max_samples=5000  # Ograniczenie dla szybszego przetwarzania

In [ ]:
latent_vectors = extract_latent_vectors(
    model=model,
    dataloader=DataLoader(train_data, batch_size=64),
    device=device,
    max_samples=5000  
)

print(f"Wyekstraktowano {latent_vectors.shape[0]} wektorów latentnych")

Klasteryzacja i wizualizacja

In [ ]:
kmeans, embedding, silhouette_score = cluster_and_visualize(
    latent_vectors=latent_vectors,
    n_clusters=10,
    n_samples_umap=2000,
    experiment=experiment
)

Zapis modelu i zakończenie

In [ ]:
torch.save(model.state_dict(), "autoencoder.pth")
experiment.log_model("autoencoder", "autoencoder.pth")

print("✅ Model zapisany!")
print("📊 Eksperyment zakończony - sprawdź wyniki w Comet ML")

experiment.end()